In [23]:
from IPython.core.debugger import set_trace
from pprint import pprint
import itertools
 

def degree_of_set(j, U):
    return max(u[j] for u in U)
    
def degree(j):
    return u[j]
    
class My_Multiplier:
    def __init__(self, M):
        self.degrees = {}
        # no of variables
        n = len(M[0])
        for i in range(n):
            self.degrees[i] = 0
            for m in M:
                self.degrees[i] = max(self.degrees[i], m[i])
        self.max_degree = max (self.degrees.values())
        self.mults = {}
        for m in M:
            self.mults[tuple(m)] = set()
        self.alphas = {}
        for m in M:
            if m[n-1] == self.degrees[n-1]:
                self.mults[m].add(n-1)
        tuples = [ele for ele in itertools.product(range(0, self.max_degree+1), repeat = 1)]
        self.alphas = {}
        for t in tuples:
            self.alphas[t] = set([m for m in M if all(m[n-1] == _t for _t in t)])
        to_delete = [_ for _ in self.alphas if not self.alphas[_]]
        for t in to_delete:
            del self.alphas[t]

        for j in reversed(range(n-1)):
            for m in M:
                if m[j] == self.degrees[j]:
                    self.mults[m].add(j)
            for m in M:
                for a in self.alphas:
                    if m in self.alphas[a] and m[j] == degree_of_set(j, self.alphas[a]):
                        self.mults[m].add(j)
            if n != 0:
                tuples = [ele for ele in itertools.product(range(0, self.max_degree+1), repeat = n - j)]       
                self.alphas = {}
                for t in tuples:
                    self.alphas[t] = set(m for m in M if list(m[j:]) == list(t))
                to_delete = [_ for _ in self.alphas if not self.alphas[_]]                    
                for t in to_delete:
                    del self.alphas[t]
    
M=[(1, 1, 3),
 (0, 1, 3),
 (2, 1, 3),
 (1, 2, 3),
 (0, 2, 3),
 (2, 1, 1),
 (1, 1, 2),
 (2, 1, 2),
 (0, 1, 2),
 (1, 1, 1),
 (0, 1, 1),
 (3, 0, 3),
 (3, 1, 3),
 (2, 2, 3),
 (3, 1, 2),
 (3, 1, 1)]


In [24]:
M1=[(0,1,1), (0,2,0), (1,0,0)]
mymult = My_Multiplier(M1)
pprint(mymult.mults)

{(0, 1, 1): {0, 1, 2}, (0, 2, 0): {0, 1}, (1, 0, 0): {0}}


In [25]:
mymult = My_Multiplier(M)
pprint(mymult.mults)

{(0, 1, 1): {1},
 (0, 1, 2): {1},
 (0, 1, 3): {2},
 (0, 2, 3): {1, 2},
 (1, 1, 1): {1},
 (1, 1, 2): {1},
 (1, 1, 3): {2},
 (1, 2, 3): {1, 2},
 (2, 1, 1): {1},
 (2, 1, 2): {1},
 (2, 1, 3): {2},
 (2, 2, 3): {0, 1, 2},
 (3, 0, 3): {0, 2},
 (3, 1, 1): {0, 1},
 (3, 1, 2): {0, 1},
 (3, 1, 3): {0, 2}}


In [26]:
M2=[(2,2,3), (3,0,3), (3,1,1), (0,1,1)]
m2=My_Multiplier(M2)
pprint(m2.mults)

{(0, 1, 1): {1}, (2, 2, 3): {0, 1, 2}, (3, 0, 3): {0, 2}, (3, 1, 1): {0, 1}}


In [27]:
M4=[(0,0,0,1,1),(0,0,1,0,1),(0,1,0,0,1),(0,0,0,2,0),(0,0,1,1,0),(0,0,2,0,0)]
m4=My_Multiplier(M4)
pprint(m4.mults)

{(0, 0, 0, 1, 1): {0, 1, 2, 3, 4},
 (0, 0, 0, 2, 0): {0, 1, 2, 3},
 (0, 0, 1, 0, 1): {0, 1, 2, 4},
 (0, 0, 1, 1, 0): {0, 1, 2},
 (0, 0, 2, 0, 0): {0, 1, 2},
 (0, 1, 0, 0, 1): {0, 1, 4}}


In [32]:
def complete(S):
    # S: list of monomial tuples. Highest variable ist the last index 
    # highest index is most important, we, have xn >...> x1
    monomials = S[:]
    result = []
    all_vars = set([_ for _ in range(len(monomials[0]))])
    while 1:
        m0 = []
        # multiplier-collection is our M
        multiplier_collection = []
        division = My_Multiplier(monomials)
        multiplier_collection = [(v, all_vars - v, k) for k, v in division.mults.items()]
        for mult, nonmult, monom, in multiplier_collection:
            for n in nonmult:
                _m0 = list(monom)
                _m0[n] += 1
                m0.append((_m0, n, monom))
        to_remove = []
        #set_trace()
        for _m0 in m0:
            # S3: check whether in class of any of the monomials
            for mult, nonmult, monom in multiplier_collection:
                s1 = [_m0[0][x] >= monom[x] for x in mult]
                s2 = [_m0[0][x] == monom[x] for x in nonmult]
                if all(s1) and all(s2):
                    # this is in _m0's class
                    to_remove.append(_m0)
                    break
                    
        for _to in to_remove:
            try:
                m0.remove(_to)
            except:
                pass
        if not m0:
            # XXX create the _Differntial Polynomials here
            return result
        else:
            result.extend(m0)
            monomials.extend([tuple(_[0]) for _ in m0])

complete(M4)

[]

In [33]:
M5=[(2,2,3), (3,0,3), (3,1,1), (0,1,1)]
v=complete(M5); v

[([3, 1, 3], 1, (3, 0, 3)),
 ([3, 1, 2], 2, (3, 1, 1)),
 ([1, 1, 1], 0, (0, 1, 1)),
 ([0, 1, 2], 2, (0, 1, 1)),
 ([2, 1, 1], 0, (1, 1, 1)),
 ([1, 1, 2], 2, (1, 1, 1)),
 ([1, 1, 2], 0, (0, 1, 2)),
 ([0, 1, 3], 2, (0, 1, 2)),
 ([2, 1, 2], 2, (2, 1, 1)),
 ([2, 1, 2], 0, (1, 1, 2)),
 ([1, 1, 3], 2, (1, 1, 2)),
 ([1, 1, 3], 0, (0, 1, 3)),
 ([0, 2, 3], 1, (0, 1, 3)),
 ([2, 1, 3], 2, (2, 1, 2)),
 ([2, 1, 3], 0, (1, 1, 3)),
 ([1, 2, 3], 1, (1, 1, 3)),
 ([1, 2, 3], 0, (0, 2, 3))]

In [34]:
m5=My_Multiplier([tuple(_[0]) for _ in v]  + M5)

In [35]:
pprint(m5.mults)

{(0, 1, 1): {1},
 (0, 1, 2): {1},
 (0, 1, 3): {2},
 (0, 2, 3): {1, 2},
 (1, 1, 1): {1},
 (1, 1, 2): {1},
 (1, 1, 3): {2},
 (1, 2, 3): {1, 2},
 (2, 1, 1): {1},
 (2, 1, 2): {1},
 (2, 1, 3): {2},
 (2, 2, 3): {0, 1, 2},
 (3, 0, 3): {0, 2},
 (3, 1, 1): {0, 1},
 (3, 1, 2): {0, 1},
 (3, 1, 3): {0, 2}}


In [36]:
new=set(m5.mults.keys()) - set(M5)

In [37]:
new

{(0, 1, 2),
 (0, 1, 3),
 (0, 2, 3),
 (1, 1, 1),
 (1, 1, 2),
 (1, 1, 3),
 (1, 2, 3),
 (2, 1, 1),
 (2, 1, 2),
 (2, 1, 3),
 (3, 1, 2),
 (3, 1, 3)}